# Optimization with Gradient Descent

Improving our neural network by optimizing Gradient Descent

Backpropagation allowed us to measure how each weight in the network contributed to the overall error. This ultimately allowed us to change these weights using a different algorithm, **Gradient Descent**. 

The takeaway here is that **backpropagation doesn't optimize**! It moves the error information from the end of the network to all the weights inside the network so that a different algorithm can optimize those weights to fit our data. We actually have a plethora of different **nonlinear optimization methods** that we could use with backpropagation:

**A Few Optimization Methods:**
* [Annealing](http://auai.org/uai2015/proceedings/papers/73.pdf)
* [Stochastic Gradient Descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)
* [AW-SGD](https://arxiv.org/pdf/1506.09016v1.pdf)
* [Momentum (SGD)](http://ruder.io/optimizing-gradient-descent/index.html#momentum)
* [Nesterov Momentum (SGD)](http://proceedings.mlr.press/v28/sutskever13.pdf)
* [AdaGrad](http://ruder.io/optimizing-gradient-descent/index.html#adagrad)
* [AdaDelta](http://ruder.io/optimizing-gradient-descent/index.html#adadelta)
* [ADAM](http://ruder.io/optimizing-gradient-descent/index.html#adam)
* [BFGS](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm)
* [LBFGS](https://en.wikipedia.org/wiki/Limited-memory_BFGS)


Many of these optimizations are good for different purposes, and in some cases several can be used together. In this tutorial, we will walk through Gradient Descent, which is arguably the simplest and most widely used neural network optimization algorithm.

# Gradient Descent

Imagine that you had a red ball inside of a rounded bucket like in the picture below.  
Imagine further that the red ball is trying to find the bottom of the bucket.  
This is **optimization**. In our case, the ball is optimizing it's position (from left to right) to find the lowest point in the bucket.

So, to gamify this a bit. The ball has two options, left or right. It has one goal, get as low as possible.  
So, it needs to press the left and right buttons correctly to find the lowest spot

![](../images/sgd-description-001.png)

So, what information does the ball use to adjust its position to find the lowest point? The only information it has is the **slope** of the side of the bucket at its current position, pictured below with the blue line.  
Notice that when the slope is negative (downward from left to right), the ball should move to the right. However, when the slope is positive, the ball should move to the left. As you can see, this is more than enough information to find the bottom of the bucket in a few iterations. This is a sub-field of optimization called gradient optimization. (Gradient is just a fancy word for slope or steepness). 

![](../images/sgd-description-002.png)

**Oversimplified Gradient Descent:**

* Calculate slope at current position
* If slope is negative, move right
* If slope is positive, move left
* (Repeat until slope == 0)

The question is, however, *how much should the ball move* at each time step? Look at the bucket again.  
The steeper the slope, the farther the ball is from the bottom. That's helpful! 

Let's improve our algorithm to leverage this new information. Also, let's assume that the bucket is on an (x,y) plane.  
So, it's location is x (along the bottom). Increasing the ball's "x" position moves it to the right.  
Decreasing the ball's "x" position moves it to the left.

**Naive Gradient Descent:**
* Calculate "slope" at current "x" position
* Change x by the negative of the slope. (x = x - slope)
* (Repeat until slope == 0)

This is a considerable improvement to our algorithm. For very positive slopes, we move left by a lot.  
For only slightly positive slopes, we move left by only a little. As it gets closer and closer to the bottom, it takes smaller and smaller steps until the slope equals zero, at which point it stops. This stopping point is called convergence.

# Sometimes It Breaks

Gradient Descent isn't perfect. Let's take a look at its issues and how people get around them.  
This will allow us to improve our network to overcome these issues.

## Problem 1: When slopes are too big

How big is too big? Remember our step size is based on the steepness of the slope. Sometimes the slope is so steep that we overshoot by a lot. Overshooting by a little is ok, but sometimes we overshoot by so much that we're even farther away than we started! See below. 

![](../images/sgd-description-003.png)

What makes this problem so destructive is that overshooting this far means we land at an EVEN STEEPER slope in the opposite direction. This causes us to overshoot again EVEN FARTHER. This viscious cycle of overshooting leading to more overshooting is called divergence.

## Solution 1: Make Slopes Smaller

Lol. This may seem too simple to be true, but it's used in pretty much every neural network. If our gradients are too big, we make them smaller! We do this by multiplying them (all of them) by a single number between 0 and 1 (such as 0.01). This fraction is typically a single float called **alpha**. When we do this, we don't overshoot and our network converges. 

![](../images/sgd-description-004.png)

**Improved Gradient Descent:**  
alpha = 0.1 (or some number between 0 and 1)
* Calculate "slope" at current "x" position
* x = x - (alpha*slope)
* (Repeat until slope == 0)


## Problem 2: Local Minimums

Sometimes your bucket has a funny shape, and following the slope doesn't take you to the absolute lowest point.  
Consider the picture below.

![](../images/sgd-description-005.png)

This is by far the most difficult problem with gradient descent. There are a myriad of options to try to overcome this. Generally speaking, they all involve an element of random searching to try lots of different parts of the bucket. 

## Solution 2: Multiple Random Starting States

There are a myriad of ways in which randomness is used to overcome getting stuck in a local minimum. It begs the question, if we have to use randomness to find the global minimum, why are we still optimizing in the first place? Why not just try randomly? The answer lies in the graph below. 

![](../images/sgd-description-006.png)

Imagine that we randomly placed 100 balls on this line and started optimizing all of them. If we did so, they would all end up in only 5 positions, mapped out by the five colored balls above. The colored regions represent the domain of each local minimum. For example, if a ball randomly falls within the blue domain, it will converge to the blue minimum. This means that to search the entire space, we only have to randomly find 5 spaces! This is far better than pure random searching, which has to randomly try EVERY space (which could easily be millions of places on this black line depending on the granularity). 

**In Neural Networks**: One way that neural networks accomplish this is by having very large hidden layers. You see, each hidden node in a layer starts out in a different random starting state. This allows each hidden node to converge to different patterns in the network. Parameterizing this size allows the neural network user to potentially try thousands (or tens of billions) of different local minima in a single neural network.

**Sidenote 1: This is why neural networks are so powerful!** They have the ability to search far more of the space than they actually compute! We can search the entire black line above with (in theory) only 5 balls and a handful of iterations. Searching that same space in a brute force fashion could easily take orders of magnitude more computation.

**Sidenote 2**: A close eye might ask, "Well, why would we allow a lot of nodes to converge to the same spot? That's actually wasting computational power!" That's an excellent point. The current state-of-the-art approaches to avoiding hidden nodes coming up with the same answer (by searching the same space) are Dropout and Drop-Connect.


## Problem 3: When Slopes are Too Small

Neural networks sometimes suffer from the slopes being too small.  
The answer is also obvious but I wanted to mention it here to expand on its symptoms. Consider the following graph.

![](../images/sgd-description-007.png)

Our little red ball up there is just stuck! If your alpha is too small, this can happen. The ball just drops right into an instant local minimum and ignores the big picture. It doesn't have the **umph** to get out of the rut.

![](../images/sgd-description-008.png)

And perhaps the more obvious symptom of deltas that are too small is that the convergence will just take a very, very long time. 

## Solution 3: Increase the Alpha

As you might expect, the solution to both of these symptoms is to increase the alpha. We might even multiply our deltas by a weight higher than 1. This is very rare, but it does sometimes happen.

# SGD in Neural Networks

So at this point you might be wondering, how does this relate to neural networks and backpropagation? This is the hardest part, and also quite important.

![](../images/sgd-description-009.png)

That big nasty curve? In a neural network, we're trying to minimize the error with respect to the weights. So, what that curve represents is the network's error relative to the position of a single weight.

So, if we computed the network's error for every possible value of a single weight, it would generate the curve you see above. We would then pick the value of the single weight that has the lowest error (the lowest part of the curve). Single weight because it's a two-dimensional plot. Thus, the x dimension is the value of the weight and the y dimension is the neural network's error when the weight is at that position.

Let's take a look at what this process looks like in a simple 2 layer neural network.

# 2 Layer Neural Network:

In [1]:
import numpy as np

# compute sigmoid nonlinearity
def sigmoid(x, deriv=False):
    if deriv is True :
        return x*(1-x)
    return 1/(1 + np.exp(-x))
    
# input dataset
X = np.array([[0, 1],
              [0, 1],
              [1, 0],
              [1, 0]])
    
# output dataset            
y = np.array([[0, 0, 1, 1]]).T

# seed random numbers to make calculation
# deterministic (just a good practice)
np.random.seed(1)

# initialize weights randomly with mean 0
synapse_0 = 2*np.random.random((2,1)) - 1

for iter in range(10000):
    # forward propagation
    layer_0 = X
    layer_1 = sigmoid(np.dot(layer_0,synapse_0))

    # how much did we miss?
    layer_1_error = layer_1 - y

    # multiply how much we missed by the 
    # slope of the sigmoid at the values in layer_1
    layer_1_delta = layer_1_error * sigmoid(layer_1, True)
    synapse_0_derivative = np.dot(layer_0.T, layer_1_delta)

    # update weights
    synapse_0 -= synapse_0_derivative

print("Output After Training:")
print(layer_1)

Output After Training:
[[ 0.00505119]
 [ 0.00505119]
 [ 0.99494905]
 [ 0.99494905]]


So, in this case, we have a single error at the output (single value), which is computed with `layer_1_error = layer_1 - y`. Since we have 2 weights, the output "error plane" is a 3 dimensional space. We can think of this as an (x,y,z) plane, where vertical is the error, and x and y are the values of our two weights in syn0.

Let's try to plot what the error plane looks like for the network/dataset above. So, how do we compute the error for a given set of weights?

```python
layer_0 = X
layer_1 = sigmoid(np.dot(layer_0,synapse_0))

layer_1_error = layer_1 - y
```

If we take that logic and plot the overall error (a single scalar representing the network error over the entire dataset) for every possible set of weights (from -10 to 10 for x and y), it looks something like this.

![](../images/sgd-description-010.gif)

Don't be intimidated by this. It really is as simple as computing every possible set of weights, and the error that the network generates at each set. x is the first synapse_0 weight and y is the second synapse_0 weight. z is the overall error. As you can see, our output data is **positively correlated** with the first input data. Thus, the error is minimized when x (the first synapse_0 weight) is high. What about the second synapse_0 weight? How is it optimal?

#### How Our 2 Layer Neural Network Optimizes

So, given code above end up computing the error. It can be natural to see that the following code optimize to reduce the error. This is where Gradient Descent is happening! Remember our pseudocode? 

```python
(1) layer_1_delta = layer_1_error * sigmoid(layer_1, True)
(2) synapse_0_derivative = np.dot(layer_0.T, layer_1_delta)

(3) synapse_0 -= synapse_0_derivative
```

**Naive Gradient Descent:**
* (1) and (2): Calculate "slope" at current "x" position
* (3): Change x by the negative of the slope. (x = x - slope)
* (Repeat until slope == 0)

It's exactly the same thing! The only thing that has changed is that we have 2 weights that we're optimizing instead of just 1. The logic, however, is identical.

Source: [A Neural Network in 13 lines of Python](http://iamtrask.github.io/2015/07/27/python-network-part2/)